# Authentication

Read secrets

In [18]:
CLIENT_ID = "98cdd6d612d54a6fa2b0f9e65f48a323"
CLIENT_SECRET = "d5550859d62640eaacd12d86c8d0d351"

Request token

- url: The endpoint of the API
- headers: Information for the HTTP request, such as authentication
- data: The payload of the request - all other information such as requested IDs, queries, ...

In [19]:
import requests
import base64

url = "https://accounts.spotify.com/api/token"


credentialsString = f"{CLIENT_ID}:{CLIENT_SECRET}" # string
credentialsBytes = credentialsString.encode("utf-8") # bytes
credentialsBytesBase64 = base64.b64encode(credentialsBytes) # bytes base64 encoded
credentialsStringBase64 = credentialsBytesBase64.decode("utf-8") # string base64 encoded

headers = {"Authorization": f"Basic {credentialsStringBase64}"}
data = {"grant_type": "client_credentials"}

r = requests.post(url, headers=headers, data=data)

# Extract the token
token = r.json()["access_token"]
token


'BQDcCcfWEz1IKPaJiidgua-OJgCWzMrT1cM3s8ZzFZIaTYnVku38wwqEWZE_A99RncWelDNgTHFlEftg3DKMt13vwf-sJ-5ltQwnyIDVQ8IHrgziiTs'

# Getting Track Information

In this section we will utilize the [GET /playlist/{playlist_id}/tracks](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-playlists-tracks) endpoint

For this endpoint, the spotify API requests all information in the url except for the authorization which remains in the header

The url we will use is https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields={fields}

Basics for urls 
- Protocol. http:// or https://
    - https:// ensures encrypton of the traffic between client and surver using SSL/TLS. Never enter your password in a http:// website, as it will be sent unencrypted through the network (e.g. WLAN)
- Domain. The identifying part. A Top-level domain (.com) is split with a (.) from the domain name (spotify). 
    - spotify.com: Only one entity can control spotify.com so we know we can trust the URL
    - Before entering your password check for the domain, it is the only part that can not be manipulated
- Sub-Domain. Everything before the second last (.): here (api.). It helps developers seperating traffic to different servers. Spotify uses different sub-domains
    - api.spotify.com for the API
    - open.spotify.com for the webplayer
    - developer.spotify.com for documentation
    - www.spotify.com is a subdomain too, it redirects to open.spotify.com
- Routes. Everything between the first (/) and the (?). On a server (e.g. api.spotify.com) it describes where to find the resource. 
    - /v1: Version 1 of the API
    - /v1/playlists: We are requesting playlist information
    - /v1/playlists/{playlist_id}: Spotify provides a route for each playlist based on the playlist id. The curly brackets indicate a placeholder
    - /v1/playlists/{playlist_id}/tracks: We are requesting track information for that playlist
- Variables. Everything after the (?).
    - fields: A variable used by spotify to specify which information should be returned to the client

In [5]:
playlist_id = "1qvW13XhfMMZMlzQx362HR"
fields = "next,items(track(artists(id,name),album(id,name),id,name))"

playlist_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields={fields}"

headers = {"Authorization": "Bearer " + token}

# Get the playlist's tracks informations
r = requests.get(url=playlist_url, headers=headers)


The response object contains the result of our request. As no error has been trown, we know that the request reached the server, we got a response back and had no connectivity issues. Howerever, the response could have been unsuccessful, e.g., with a status code 401: Unauthorized or 404: Not Found

In [6]:
print(r.status_code)
print(r.content)

200
b'{\n  "items" : [ {\n    "track" : {\n      "album" : {\n        "id" : "0c6GZXLOQNRRfoJSCraLwc",\n        "name" : "Pendulum"\n      },\n      "artists" : [ {\n        "id" : "0tquhVod10o2zf1ht2aVoz",\n        "name" : "Elior"\n      }, {\n        "id" : "5MvvhhTGyd2iGzaksZpLEt",\n        "name" : "eaup"\n      } ],\n      "id" : "5XfGPtiNkoPkeK00QTEHCb",\n      "name" : "Pendulum"\n    }\n  }, {\n    "track" : {\n      "album" : {\n        "id" : "3W1TKjeKQawzmVlBGr9eOr",\n        "name" : "Cheer Up"\n      },\n      "artists" : [ {\n        "id" : "5MvvhhTGyd2iGzaksZpLEt",\n        "name" : "eaup"\n      } ],\n      "id" : "0CVT6rpAdkDm4NKLdv7E9g",\n      "name" : "Cheer Up"\n    }\n  }, {\n    "track" : {\n      "album" : {\n        "id" : "04YdOVml7kXJN985zOOHPX",\n        "name" : "Endless Road"\n      },\n      "artists" : [ {\n        "id" : "2RBJhPBOr8mI5evwaL4HR3",\n        "name" : "Ghostnaut"\n      }, {\n        "id" : "1XFN3VcuKr4tsTtQlRiTgK",\n        "name" : "Mond

We can parse the content in a python dictionary using the .json() method

In [7]:
r_json = r.json()
r_json

{'items': [{'track': {'album': {'id': '0c6GZXLOQNRRfoJSCraLwc',
     'name': 'Pendulum'},
    'artists': [{'id': '0tquhVod10o2zf1ht2aVoz', 'name': 'Elior'},
     {'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}],
    'id': '5XfGPtiNkoPkeK00QTEHCb',
    'name': 'Pendulum'}},
  {'track': {'album': {'id': '3W1TKjeKQawzmVlBGr9eOr', 'name': 'Cheer Up'},
    'artists': [{'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}],
    'id': '0CVT6rpAdkDm4NKLdv7E9g',
    'name': 'Cheer Up'}},
  {'track': {'album': {'id': '04YdOVml7kXJN985zOOHPX', 'name': 'Endless Road'},
    'artists': [{'id': '2RBJhPBOr8mI5evwaL4HR3', 'name': 'Ghostnaut'},
     {'id': '1XFN3VcuKr4tsTtQlRiTgK', 'name': 'Mondo Loops'},
     {'id': '773SnwBYAvo9JlHuulUvMu', 'name': 'Lunath'}],
    'id': '1Kqx9kj7twQEexWCa4WHUZ',
    'name': 'Endless Road'}},
  {'track': {'album': {'id': '4WjETwIOQcthrTokTKIOvL', 'name': 'II'},
    'artists': [{'id': '6CzaYsf8RIrCHNDhFGSfgm', 'name': 'Havana Swim Club'},
     {'id': '5YNnmXnEEEJM0ArqHDSyET

In [8]:
items = r_json["items"]
next = r_json["next"]

Spotify returns only 100 tracks per request

In [9]:
print(len(items))
print(next)

100
https://api.spotify.com/v1/playlists/1qvW13XhfMMZMlzQx362HR/tracks?offset=100&limit=100&fields=next,items(track(artists(id,name),album(id,name),id,name))


Again we need to account for pagination. 

In [10]:
all_items = items

r = requests.get(url=next, headers=headers)
r_json = r.json()
items = r_json["items"]
next = r_json["next"]

all_items = all_items + items
all_items

[{'track': {'album': {'id': '0c6GZXLOQNRRfoJSCraLwc', 'name': 'Pendulum'},
   'artists': [{'id': '0tquhVod10o2zf1ht2aVoz', 'name': 'Elior'},
    {'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}],
   'id': '5XfGPtiNkoPkeK00QTEHCb',
   'name': 'Pendulum'}},
 {'track': {'album': {'id': '3W1TKjeKQawzmVlBGr9eOr', 'name': 'Cheer Up'},
   'artists': [{'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}],
   'id': '0CVT6rpAdkDm4NKLdv7E9g',
   'name': 'Cheer Up'}},
 {'track': {'album': {'id': '04YdOVml7kXJN985zOOHPX', 'name': 'Endless Road'},
   'artists': [{'id': '2RBJhPBOr8mI5evwaL4HR3', 'name': 'Ghostnaut'},
    {'id': '1XFN3VcuKr4tsTtQlRiTgK', 'name': 'Mondo Loops'},
    {'id': '773SnwBYAvo9JlHuulUvMu', 'name': 'Lunath'}],
   'id': '1Kqx9kj7twQEexWCa4WHUZ',
   'name': 'Endless Road'}},
 {'track': {'album': {'id': '4WjETwIOQcthrTokTKIOvL', 'name': 'II'},
   'artists': [{'id': '6CzaYsf8RIrCHNDhFGSfgm', 'name': 'Havana Swim Club'},
    {'id': '5YNnmXnEEEJM0ArqHDSyET', 'name': 'Lonely Benson'}],
  

Next is None, indicating that we have recieved all tracks in that playlist

In [11]:
print(len(all_items))
print(next)

113
None


Let's inspect the returned objects closer and extract some album and artist names

In [12]:
first_track_object = all_items[0]
first_track_object

{'track': {'album': {'id': '0c6GZXLOQNRRfoJSCraLwc', 'name': 'Pendulum'},
  'artists': [{'id': '0tquhVod10o2zf1ht2aVoz', 'name': 'Elior'},
   {'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}],
  'id': '5XfGPtiNkoPkeK00QTEHCb',
  'name': 'Pendulum'}}

In [13]:
first_track = first_track_object["track"]
first_artists = first_track["artists"]
first_album = first_track["album"]



It seems that two artists collaborated on the first returned track

In [14]:
first_artists

[{'id': '0tquhVod10o2zf1ht2aVoz', 'name': 'Elior'},
 {'id': '5MvvhhTGyd2iGzaksZpLEt', 'name': 'eaup'}]

In [13]:
import pandas as pd

columns = ["id", "name", "artist_id", "artist_name", "album_id", "album_name"]
track_df = pd.DataFrame(data=[], columns=columns)

for item in all_items:
    track = item["track"]
    track_id = track["id"]
    track_name = track["name"]

    artist = track["artists"][0]
    artist_id = artist["id"]
    artist_name = artist["name"]

    album = track["album"]
    album_id = album["id"]
    album_name = artist["name"]

    track_df = pd.concat([track_df, pd.DataFrame(data=[[track_id, track_name, artist_id, artist_name, album_id, album_name]], columns=columns)], axis=0)


track_df.set_index("id", inplace=True)
track_df

,name,artist_id,artist_name,album_id,album_name
id,,,,,
5XfGPtiNkoPkeK00QTEHCb,Pendulum,0tquhVod10o2zf1ht2aVoz,Elior,0c6GZXLOQNRRfoJSCraLwc,Elior
0CVT6rpAdkDm4NKLdv7E9g,Cheer Up,5MvvhhTGyd2iGzaksZpLEt,eaup,3W1TKjeKQawzmVlBGr9eOr,eaup
1Kqx9kj7twQEexWCa4WHUZ,Endless Road,2RBJhPBOr8mI5evwaL4HR3,Ghostnaut,04YdOVml7kXJN985zOOHPX,Ghostnaut
4ertsRlco2bn3dWqY41sxS,Fantasy,6CzaYsf8RIrCHNDhFGSfgm,Havana Swim Club,4WjETwIOQcthrTokTKIOvL,Havana Swim Club
0fO196iK1nt9tdoOLsn1pq,Ovina,14xurMk6IfCAqEzI1txycG,Prinz Leo,6tlATjYhVr3LhLZGBjtQmB,Prinz Leo
...,...,...,...,...,...
3vzauVzDj3tFhhK0LuXaid,pick up,5MvvhhTGyd2iGzaksZpLEt,eaup,7lfhClmunN9igCRTCTvCsu,eaup
0GI3utI1PXaCMiQTq1l3di,Cruisin',0tquhVod10o2zf1ht2aVoz,Elior,5jwXN0zduPh7pYUiIn1kCD,Elior
6eGpjRTgqSs8JfLPJcSSts,reality,2cZLcgAsOeknY8nJGwt5Bg,mr.neab,7zjk5CmvoP6k8M17vOrSc6,mr.neab


In [16]:
track_df[track_df["artist_name"] == "Elior"]

,name,artist_id,artist_name,album_id,album_name
id,,,,,
5XfGPtiNkoPkeK00QTEHCb,Pendulum,0tquhVod10o2zf1ht2aVoz,Elior,0c6GZXLOQNRRfoJSCraLwc,Elior
5KhoMiyDDs4vQazM6eE7jA,Calm Down,0tquhVod10o2zf1ht2aVoz,Elior,7HILhfhjOpTTtyYXMl5tqf,Elior
0pjir6f0ROoeKVFsULDrAN,winterglow,0tquhVod10o2zf1ht2aVoz,Elior,7zsXXquownEgLT97g3ZYMk,Elior
0GI3utI1PXaCMiQTq1l3di,Cruisin',0tquhVod10o2zf1ht2aVoz,Elior,5jwXN0zduPh7pYUiIn1kCD,Elior


Elior is featured 4 times in the playlist

Other endpoints to test:
- [GET /audio_featres](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)
- [GET /users/{user_id}/playlists](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-list-users-playlists). Philipp's user_id is 31spovoympggn7jtvfw3a7bbyzma